In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from PIL import Image

from scribbles_creator import *
from cellpose_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define where the images are located and what data to consider

In [5]:
folder_path = "./results/data/pred07"

In [9]:
mode = "all"
bins = [1.25]#[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]
all_suff = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
suff = ["x"] #all_suff[:5]
prediction_types = ["ilastik"] #["convpaint", "ilastik"] # ["convpaintCh1", "ilastikCh1"]

## Analyse the created scribbles and segmentations (perc. labelled vs. acc.) from saved files

In [11]:
df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'class_1_pix_gt', 'class_2_pix_gt', 'pix_labelled', 'class_1_pix_labelled', 'class_2_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'image', 'ground truth', 'scribbles', 'prediction'])

for img_num in range(0, 540, 10):
    for bin in bins:
        for s in suff:
            for pred in prediction_types:
                res = analyse_cellpose_single_file(folder_path, img_num, mode=mode, bin=bin, suff=s, pred_tag=pred, show_res=False)
                df = pd.concat([df, res], ignore_index=True)
df[:5]                

C:\Users\roman\AppData\Local\Temp\ipykernel_1328\1573255358.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, res], ignore_index=True)


,img_num,prediction type,scribbles mode,scribbles bin,suffix,class_1_pix_gt,class_2_pix_gt,pix_labelled,class_1_pix_labelled,class_2_pix_labelled,pix_in_img,perc. labelled,accuracy,image,ground truth,scribbles,prediction
0,0,ilastik,all,1.25,x,101577,94519,19188,10066,9122,196096,9.785003,0.948908,./results/data/pred07/000_img.png,./results/data/pred07/000_ground_truth.png,./results/data/pred07/000_scribbles_all_01250_...,./results/data/pred07/000_ilastik_all_01250_x.png
1,10,ilastik,all,1.25,x,120609,75487,19321,12235,7086,196096,9.852827,0.959005,./results/data/pred07/010_img.png,./results/data/pred07/010_ground_truth.png,./results/data/pred07/010_scribbles_all_01250_...,./results/data/pred07/010_ilastik_all_01250_x.png
2,20,ilastik,all,1.25,x,94288,101808,18501,8803,9698,196096,9.434665,0.934675,./results/data/pred07/020_img.png,./results/data/pred07/020_ground_truth.png,./results/data/pred07/020_scribbles_all_01250_...,./results/data/pred07/020_ilastik_all_01250_x.png
3,30,ilastik,all,1.25,x,124222,71874,19415,12615,6800,196096,9.900763,0.939994,./results/data/pred07/030_img.png,./results/data/pred07/030_ground_truth.png,./results/data/pred07/030_scribbles_all_01250_...,./results/data/pred07/030_ilastik_all_01250_x.png
4,40,ilastik,all,1.25,x,149525,46571,18490,14497,3993,196096,9.429055,0.964650,./results/data/pred07/040_img.png,./results/data/pred07/040_ground_truth.png,./results/data/pred07/040_scribbles_all_01250_...,./results/data/pred07/040_ilastik_all_01250_x.png


Analyse the comparison of convpaint models

In [2]:
folder_path = "results\data\pred06"
prediction_types = ["convpaint_l-0_s-1",
                    "convpaint_l-0_s-1-2",
                    "convpaint_l-0_s-1-2-4",
                    "convpaint_l-0_s-1-2-4-8",
                    "convpaint_l-0-2_s-1",
                    "convpaint_l-0-2_s-1-2",
                    "convpaint_l-0-2_s-1-2-4",
                    "convpaint_l-0-2_s-1-2-4-8"] # ["convpaintCh1", "ilastikCh1"]

df = pd.DataFrame(columns=['img_num', 'prediction type', 'scribbles mode', 'scribbles bin', 'suffix', 'class_1_pix_gt', 'class_2_pix_gt', 'pix_labelled', 'class_1_pix_labelled', 'class_2_pix_labelled', 'pix_in_img', 'perc. labelled', 'accuracy', 'image', 'ground truth', 'scribbles', 'prediction'])

for img_num in range(0, 540, 10):
    for pred in prediction_types:
        res = analyse_cellpose_single_file(folder_path, img_num, mode="all", bin=1.25, suff="x", pred_tag=pred, show_res=False)
        df = pd.concat([df, res], ignore_index=True)
df.shape

C:\Users\roman\AppData\Local\Temp\ipykernel_12528\1769071347.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, res], ignore_index=True)


(432, 17)

Save the data frame in a csv file:

In [12]:
from datetime import datetime
time_stamp = datetime.now().strftime("%y%m%d%H%M%S")
file_name = f"test_labels_vs_acc_{time_stamp}.csv"
df.to_csv(file_name, index=False)